### 分类评估

In [1]:
# 1.分类模型评估
    # 二分类：标注分类只有两类。正类用1表示（比较关注），负类用0表示
        # 混淆矩阵：
            # TP(Ture Positive):实际是正类，识别为正类      Y_11
            # FN(False Negative):实际是正类，识别为负类（漏）Y_10
            # FP(False Positive):实际是负类，识别为正类（错）Y_01
            # TN(True Negative):实际是负类，识别为负类      Y_00
        # 关键指标
            # 正确率：所有被正确分类所占的比例
            # 召回率：真实被分为正类的数据/所有真实的正类
            # F-score:2*召回率*准确率/召回率+准确率 
            # 查准率（Precision）：真的正类/真的正类+假的正类
            # 错误接收率（FPR）:有多少负类被划成了正类的比例 FP/(FP+TN)
            # 错误拒绝率（FRR）:真实的所有正类中有多少是被错误拒绝的（有多少真实的正类没有被判为正类） FN/(TP+FN)
    # 多类   
        # 多元混淆矩阵：
            # 正确率：所有被正确分类所占的比例
            # 召回率 和 F-score
                # 1.先计算所有的TP，FN等，再以二值方法计算 (micro方法)
                # 2.分别把每个类别当作正类，都算一个召回或者F值，然后取加权（weighted）或者不加权(macro)的平均
    # 取阈值：
        # 1.ROC曲线：在ROC中横轴代表的是FPR（错误接收率），纵轴代表的是TPR（召回率）。ROC曲线能很容易的查出任意界限值时的对性能的识别力
        # 2.AUC：是ROC曲线下的面积，其值可以直观反应ROC曲线向左上方的靠近程度
# 2.回归模型评估
# 3.聚类模型评估
# 4.关联模型评估

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [15]:
# 回归测试
def regr_test(features, label):
    print("X", features)
    print("Y", label)
    # LinearRegression 线性回归
    # Ridge 岭回归
    # Lasso
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    # 线性回归 (常用)
    # regr = LinearRegression()
    # 岭回归 （特殊情况使用）
    # regr = Ridge(alpha=0.8)
    # Lasso （特殊情况使用）
    regr = Lasso(alpha=0.004)
    
    regr.fit(features.values, label.values)
    # 预测
    Y_pred = regr.predict(features.values)
    # 参数
    print("Coef:", regr.coef_)
    # 衡量回归方程的好坏
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    print("MSE:", mean_squared_error(label.values, Y_pred))
    # 回归模型评估
    print("MAE:", mean_absolute_error(label.values, Y_pred))
    print("R2:", r2_score(label.values, Y_pred))

def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择

    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)  
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    # print(len(X_train), len(X_validation), len(X_test))
    
    # 衡量指标
    # accuracy_score 准确率
    # recall_score 召回率
    # f1_score F值
    from sklearn.metrics import accuracy_score, recall_score, f1_score
    # KNN
    # NearestNeighbors 此方法可以直接获得一个点附近最近的几个点
    from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
    # 逻辑回归
    from sklearn.linear_model import LogisticRegression
    # 提升树 GBDT
    from sklearn.ensemble import GradientBoostingClassifier
    
    
    # 人工神经网络
    # Sequential 层级容器
    from keras.models import Sequential
    # Dense 神经网络层/稠密层， Activation激活函数
    from keras.layers.core import Dense, Activation
    # 随机梯度下降算法
    from keras.optimizers import SGD 
    # 反向传播算法 PyBrain
    mdl = Sequential()
    # 创建稠密层表示输入层
    # 50 下个隐含层的神经元个数
    # input_dim  输入的维度
    mdl.add(Dense(50, input_dim=len(f_v[0])))
    # 激活函数
    mdl.add(Activation("sigmoid"))
    # 接入输出层 (上一层输出几个，下一层就会输入几个)
    mdl.add(Dense(2))
    # 保证归一化
    mdl.add(Activation("softmax"))
    # 优化器 lr 学习率
    sgd = SGD(lr=0.08)
    # 编译建立模型
    # loss 最优化函数、损失函数
    # mean_squared_error 平均平方误差
    # optimizer 使用什么样的优化器  sgd  adam(亚当优化器)，迈开步子跑 
    # nb_epoch 一共迭代的次数 
    # batch_size 随机梯度下降算法每次选取的数量。
    mdl.compile(loss="mean_squared_error", optimizer="adam")
    mdl.fit(X_train, np.array([[0, 1] if i==1 else [1, 0] for i in Y_train]), nb_epoch=1000, batch_size=8999)
    # 引入ROC曲线
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    f = plt.figure()
    
    
    
    # 进行预测和评价 【训练集，验证集，测试集】
    xy_lst = [(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)]
    for i in range(len(xy_lst)):
        # X_part 真实值
        X_part = xy_lst[i][0]
        # y_part 预测值
        Y_part = xy_lst[i][1]
        # 预测值
        Y_pred = mdl.predict(X_part)
        print(Y_pred)
        # 取其为正类的概率
        Y_pred = np.array(Y_pred[:1]).reshape((1, -1))[0]
        # 绘制图形
        f.add_subplot(1, 3, i+1)
        # 返回值有 fpr tpr  thresholds阈值
        fpr, tpr, thresholds = roc_curve(Y_part, Y_pred)
        plt.plot(fpr, tpr)
        print("NN", "AUC", auc(fpr, tpr))
        print("NN", "AUC_Score", roc_auc_score(Y_part, Y_pred))
    plt.show()
    return 

#     models = []
#     # Adaboost
#     # random_state 随机数的种子值
#     # base_estimator 弱分类的基本分类器 需要两个属性 classes_ 和 n_classes_ 默认的是决策树
#     # n_estimators 及联的决策树的数量 默认是50个
#     # learning_rate 根据0-1的数进行衰减
#     # algorithm 针对 base_estimator 进行的选择，如果base_estimator可以得到一个判别的概率，那么就可以使用SAMME.R,得到更好的效果
# #     models.append(("AdaBoost", AdaBoostClassifier(n_estimators=100)))
#     # penalty l1,l2 使用L1正则化或L2正则化
#     # tol 计算的精度
#     # C C越大，正则化因子所占的比例越小，反之越大
#     # solver 使用的方法 sag 随机平均梯度下降算法 默认是liblinear
#     # max_iter 最大的迭代次数
#     # coef_ 参数
#     # intercept_ 截距
#     models.append(("LogisticRegression", LogisticRegression(C=1000, tol=1e-10, solver="sag", max_iter=10000)))
#     # 提升树 GBDT
#     # max_depth 深度
#     # n_estimators 树的数量
#     # learning_rate 衰减比率
#     # criterion 标准
#     models.append(("GBDT", GradientBoostingClassifier(max_depth=6, n_estimators=100)))
#     for clf_name, clf in models:
#         # 先进行拟合 训练
#         clf.fit(X_train, Y_train)
#         # 进行预测和评价 【训练集，验证集，测试集】
#         xy_lst = [(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)]
#         for i in range(len(xy_lst)):
#             # X_part
#             X_part = xy_lst[i][0]
#             # y_part
#             Y_part = xy_lst[i][1]
#             # 预测值
#             Y_pred = clf.predict(X_part)
#             # 0训练集 1验证集 2测试集
#             print(i)
#             print(clf_name, "-ACC:", accuracy_score(Y_part,  Y_pred))
#             print(clf_name, "-REC:", recall_score(Y_part,  Y_pred))
#             print(clf_name, "-F-Score:", f1_score(Y_part,  Y_pred))
            
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    regr_test(features[["number_project", "average_monthly_hours"]], features["last_evaluation"])
    
#     hr_modeling(features, label)
if __name__=="__main__":
    main()

X        number_project  average_monthly_hours
0                 0.0               0.285047
1                 0.6               0.775701
2                 1.0               0.822430
3                 0.6               0.593458
4                 0.0               0.294393
...               ...                    ...
14994             0.0               0.257009
14995             0.0               0.299065
14996             0.0               0.219626
14997             0.8               0.859813
14998             0.0               0.289720

[14999 rows x 2 columns]
Y 0        0.265625
1        0.781250
2        0.812500
3        0.796875
4        0.250000
           ...   
14994    0.328125
14995    0.187500
14996    0.265625
14997    0.937500
14998    0.250000
Name: last_evaluation, Length: 14999, dtype: float64
Coef: [0.22810785 0.21537058]
MSE: 0.05993150070504495
MAE: 0.20767975383640475
R2: 0.16209792042024462


### 增益率与KS图

In [13]:
# 增益图：横轴是测试图的取样比例，纵轴是平均样本比例/平均比例 例子：作弊用户
# KS图：横轴为测试集取样比例，纵轴为TPR和FPR。关注的是两条曲线的最大差距。这个差距可以反映对正类样本的区分度

### 回归模型评估

In [16]:
# 回归模型要拟合的真实值是连续值，所以回归模型的预测值也是连续值。我们的任务就是确定这两个连续值序列的差距到底有多大
# 常用的方法：
    # 关键指标
    # MAE(Mean Absolute Error):每个预测值与真实值相减(残差)，取绝对值相加，最后求平均。（不常用）
    # MSE（Mean Square Error）:每个预测值与真实值相减(残差)的平方相加，最后求平均。（常用）
    # RMSE(Root MSE):对MSE求开根号
    # r2_score(决定系数)：SSR/SST=预测值相对与真实平均值的离散程度/真实值的离散程度
# 代码看上面

### 聚类模型评估

In [17]:
# 关键指标：
    # RMS(Root Mean Square):每个聚类的值-每个类的平均值平方再求和，再对和开根号在求均值。值越小聚类效果越好
    # 轮廓系数
        # a(i)为样本i与簇内其他样本的平均距离；
        # b(i)为样本i与其他某簇样本的平均距离，多个簇b(i)取最小
        # s(i) = b(i)-a(i)/max{a(i), b(i)}   分离度-内聚度/分离度或内聚度中较大的那一个  分离度越大越好，内聚度越小越好。
        # s(i)越趋近于1越好，越接近于-1越糟糕
# 代码看-- 6_数据挖掘建模_04_聚类 --中的代码

### 关联模型

In [ ]:
# 关键指标  可参考 6_数据挖掘建模_04_聚类
    # 支持度：可以帮助我们找到最大的频繁项集，它可形成一种规模效应，可以形成多种组合
    # 置信度：观察组合之间的关系，可以得到置信度
    # 提升度：看置信度对另外一个项集是否有提升作用